In [ ]:
!pip install llama-index

In [ ]:
!git clone https://github.com/jerryjliu/llama_index.git

In [3]:
!ls llama_index/examples/paul_graham_essay/

data			 index_with_query.json
DavinciComparison.ipynb  InsertDemo.ipynb
GPT4Comparison.ipynb	 KeywordTableComparison.ipynb
index.json		 SentenceSplittingDemo.ipynb
index_list.json		 splitting_1.txt
index_table.json	 splitting_2.txt
index_tree_insert.json	 TestEssay.ipynb


In [4]:
!cp -r llama_index/examples/paul_graham_essay/data .

In [5]:
import os
os.environ['OPENAI_API_KEY'] = "..."

In [ ]:
!pip install openai

In [7]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

In [8]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)


The author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-stabilized apartment, building an online store builder, editing Lisp expressions, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.


In [9]:
index.storage_context.persist()

In [10]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

In [11]:
response = query_engine.query("著者はどんな育ち方をしてきたのでしょうか？。出力は日本語でお願いします。")
print(response)


著者は、自分が何をすべきかを決めるために、過去にどのように決めてきたかを自分に問いかけるためにエッセイを書き始めました。彼は、自分が体験したことを他の人にも興味深く、同じような混乱した人々を勇気づけるために、より詳細なバージョンを書きました。彼は、コンピューターを使った時代に入る前に、イタリア語を学び、フィレンツェを歩いて様々な状況を体験し、また、芸術的なスキルを身につけました。彼は、インターリーブという会社を設立し、インターネット上で何かを公開するという考えを実現しました。彼は、自分のエッセイをインターネット上に公開し、誰もが読むことができるということを実感しました。そして、彼は、自分が書いたエッセイを読んだ人々から反応を得ることを学びました。彼は、自分が書くエッセイを常に行うことを決め、スパムフィルターを開発し、絵を描いたり、料理を学んだり、ビルを購入したりしながら、様々なことに取り組んできました。


In [12]:
response = query_engine.query("What did the author do growing up?Please output in Japanese.")
print(response)


作者が育ってきたときには、学校以外では短編小説を書くこととプログラミングをしていました。IBM 1401というデータ処理用のコンピュータを使ってプログラミングをしていましたが、入力データがないため、何もできなかったと言っています。その後、マイクロコンピュータが登場し、彼は自分で組み立てたHeathkitのキットを使ってプログラミングをしたり、TRS-80を使ってゲームを作ったり、モデルロケットの高さを予測するプログラムを書いたり、父親が書いた本のためのワープロを書いたりしました。


In [14]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data2').load_data()
index2 = GPTVectorStoreIndex.from_documents(documents)

In [15]:
index2.storage_context.persist('index_kenpou')

In [18]:
query_engine2 = index2.as_query_engine()


In [19]:
response = query_engine2.query("国民の持つ権利で重要なものをリストであげてください。それが何条に書かれているかも示してください。")
print(response)


1. 生命、自由及び幸福追求に対する国民の権利 (第一一条)
2. 個人としての尊重 (第一三条)
3. 国民の不断の努力によつて、基本的人権を保持する義務 (第一二条)
4. 公共の福祉のために基本的人権を利用する責任 (第一二条)
5. 正当に選挙された国会における代表者を通じて行動する権利 (前文)
6. 国民が主権を持つことを宣言する権利 (前文)
7. 平和を愛する諸国民の公正と信義に信頼して安全と生存を保持する権利 (前文)
8. 専制と隷従、圧迫と偏狭を地上から永遠に除去する権利 (前文)


In [21]:
from llama_index import LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size=512)

index3 = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine3 = index3.as_query_engine(service_context=service_context)

In [22]:
response = query_engine3.query("国民のもつ権利をリストしてください")
print(response)

- 生命、自由及び幸福追求に対する国民の権利については、公共の福祉に反しない限り、立法その他の国政の上で、最大の尊重を必要とする。
- すべて国民は、法の下に平等であつて、人種、信条、性別、社会的身分又は門地により、政治的、経済的又は社会的関係において、差別されない。
- 華族その他の貴族の制度は、これを認めない。
- 栄誉、勲章その他の栄典の授与は、いかなる特権も伴はない。栄典の授与は、現にこれを有し、又は将来これを受ける者の一代に限り、その効力を有する。
- 公務員を選定し、及びこれを罷免することは、国民固有の権利である。
- すべて公務員は、全体の奉仕者であつて、一部の奉仕者ではない。


In [23]:
from llama_index import download_loader

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")

loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=['https://www.aozora.gr.jp/cards/001528/files/474.html'])

In [24]:
indexw = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine_w = indexw.as_query_engine(service_context=service_context)

In [25]:
response = query_engine_w.query("要約をお願いします。")
print(response)

この文書は、日本国憲法の第53条と第54条について説明しています。第53条では、内閣が臨時国会の召集を決定できることが示されています。第54条では、衆議院が解散された場合、40日以内に衆議院議員の総選挙を行い、その選挙の日から30日以内に国会を召集しなければならないことが示されています。また、参議院は衆議院が解散された場合には同時に閉会となりますが、内閣は緊急の必要がある場合には参議院の緊急集会を求めることができます。ただし、その措置は臨時のものであり、衆議院の同意がない場合には、次の国会開会の後10日以内にその効力を失うことになっています。最後に、この文書は、日本国憲法が昭和22年5月3日に施行されたことを示しています。


In [31]:
from pathlib import Path
from llama_index import download_loader

CJKPDFReader = download_loader("CJKPDFReader")

loader = CJKPDFReader()
documents = loader.load_data(file=Path('./attention.pdf'))

In [32]:
index_pdf = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine_pdf = index_pdf.as_query_engine(service_context=service_context)

In [35]:
response = query_engine_pdf.query("Please summarize this .Please explain it in a way that even a child can understand.")
print(response)

This is a research paper about a new way to make computers understand and translate languages. They made a special kind of computer program called a Transformer that uses attention to understand sentences and translate them. This new program is better than the old ones because it can work faster and make fewer mistakes. They tested it by translating English to German and French, and it did a really good job!


In [36]:
response = query_engine_pdf.query("Explain about attention heads. Please explain it in a way that even a child can understand.")
print(response)


Attention heads are like little helpers in a computer program that help the program understand sentences better. They look at different parts of the sentence and try to figure out what's important. Just like how we might focus on different words when we read a sentence, attention heads focus on different parts of the sentence to help the program understand it better. By doing this, the program can do things like translate sentences from one language to another or answer questions about a text.
